In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras

# 1. Load Processed Data (Saved in Step 2)
input_folder = '../data/processed'

# Load Features (X)
X_train = np.load(f'{input_folder}/X_train.npy')
X_val = np.load(f'{input_folder}/X_val.npy')
X_test = np.load(f'{input_folder}/X_test.npy')

# Load Targets (y)
y_train = np.load(f'{input_folder}/y_train.npy')
y_val = np.load(f'{input_folder}/y_val.npy')
y_test = np.load(f'{input_folder}/y_test.npy')

# Load Class Names (for reporting)
class_names = np.load(f'{input_folder}/class_names.npy', allow_pickle=True)

print(f"✅ Data Loaded Successfully!")
print(f"Training Data Shape: {X_train.shape}")
print(f"Number of Classes: {len(class_names)} {class_names}")


✅ Data Loaded Successfully!
Training Data Shape: (629136, 24)
Number of Classes: 7 ['A' 'B' 'C' 'D' 'E' 'F' 'G']


In [ ]:
import os
import tensorflow as tf
from tensorflow import keras

model_folder = '../models'
os.makedirs(model_folder, exist_ok=True) # Klasör yoksa oluşturur

model_path = f'{model_folder}/best_ann_model.h5'

print(f"💾 Model şuraya kaydedilecek: {model_path}")

#Model mimarisi tanımlama
model = keras.Sequential([
    # Giriş Katmanı + 1. Gizli Katman
    keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    
    # Dropout (Overfitting'i engellemek için nöronların %30'unu rastgele kapat)
    keras.layers.Dropout(0.3),
    
    # 2. Gizli Katman
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dropout(0.3),
    
    # 3. Gizli Katman (Daha derin öğrenme için)
    keras.layers.Dense(32, activation='relu'),
    
    # Çıkış Katmanı (Sınıf sayısı kadar nöron, Softmax ile olasılık hesabı)
    keras.layers.Dense(len(class_names), activation='softmax')
])

# 3. Modeli Derle (Compile)
model.compile(
    optimizer='adam',  
    loss='categorical_crossentropy', # Çoklu sınıflandırma için standart kayıp fonksiyonu
    metrics=['accuracy']
)

model.summary()


#Early Stopping: Gelişme durursa eğitimi erken bitir (Boşa vakit harcama)
early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,               # 10 tur boyunca iyileşme olmazsa dur
    restore_best_weights=True, # Durduğunda en iyi ağırlıkları geri yükle
    verbose=1
)

#Model Checkpoint: EN İYİ modeli diske kaydet
checkpoint_cb = keras.callbacks.ModelCheckpoint(
    filepath=model_path,
    save_best_only=True,      # Sadece rekor kıran modelleri kaydet
    monitor='val_loss',       # Validasyon kaybını izle
    mode='min',               # Kayıp ne kadar AZ ise o kadar iyidir
    verbose=1
)

print("\nTraining Starting...")
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=100,          
    batch_size=32,       
    callbacks=[early_stopping, checkpoint_cb], # Listeye checkpoint'i de ekledik
    verbose=1
)